In [33]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os 
import cv2 as cv
import torch
from torch import nn
from torch.optim import AdamW
from torch.utils.data import Dataset,DataLoader
import matplotlib.pyplot as plt

In [34]:
class Cfg:
    tr_batch=16
    val_batch=32
    lr=10**-3
    acc_steps=4
    betas=(.9,.999)
    eps=10**-6
    epochs=50
    

In [35]:
# transform=ToTensor()  it converts each pixel value bw 0 and 1

In [36]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader,Dataset
from torchvision import transforms
from torch.optim import lr_scheduler
from torch import optim
from torchvision.transforms import ToTensor


In [37]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader,Dataset
from torch import optim
from torchvision import transforms
from torch.optim import lr_scheduler
from PIL import Image

In [38]:
train_df=pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test_df=pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
y=train_df['label'].values
x=train_df.iloc[:,1:].values

In [39]:
train_df['fold']=0

In [40]:
from sklearn.model_selection import StratifiedKFold
skf=StratifiedKFold(n_splits=5)
for i,(t,v) in enumerate(skf.split(x,y)):
    train_df.loc[v,'fold']=i
train_df['fold']=train_df['fold'].astype('int')
    

In [41]:
x=(x/255.0)

In [42]:
x=x.reshape(-1,28,28,1)

In [43]:

plt.imshow(x[0])

In [44]:
y.shape

In [45]:
x=x[0,:].astype(np.uint8).reshape(28,28,1)
x.shape

In [ ]:
class TrainDataset(Dataset):
    def __init__(self,df):
        self.df=df
    def __getitem__(self,item):
        x=self.df.iloc[item,1:-1].values.astype(np.uint8).reshape(28,28)[:,:,None]
        y=self.df.iloc[item,0:1].values

        return x,y
    def __len__(self):
        return len(self.df)
    

class TestDataset(Dataset):
    def __init__(self,df):
        self.df=df
    def __getitem__(self,item):
        x=self.df.iloc[item,0:].values.astype(np.float16).reshape(28,28)[:,:,None]
        x=x/255.0
        #y=self.df.iloc[item,0]
        
        return x
    def __len__(self):
        return len(self.df)


        

In [ ]:
def train_fn(model,train_loader,criterion,optimizer):
    device='cuda'
    model.train()
    model.to(device)
    losses=AverageMeter()
    scaler=torch.cuda.amp.GradScaler()
    preds=[]
    labels=[]
    for i,(x,y) in tqdm_notebook(enumerate(train_loader),total=len(train_loader)):
        x=x.reshape(-1,1,28,28)
        x=x.float()
    
        x=x.to(device)
        y=y.to(device)
        
        with torch.cuda.amp.autocast():
            y_preds=model(x)
        loss=criterion(y_preds,y.view(-1,1).flatten())
        scaler.scale(loss).backward()
        losses.update(loss.item(),Cfg.tr_batch)
        if i%Cfg.acc_steps==0 or i==len(train_loader)-1:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            #scheduler.step()
        y_preds=torch.nn.functional.softmax(y_preds,dim=1)
        y_preds=torch.argmax(y_preds,dim=1)
        y_preds=y_preds.detach().cpu().numpy()
        y=y.flatten().detach().cpu().numpy()
        preds.append(y_preds)
        labels.append(y)
    pred=np.concatenate(preds)
    labels=np.concatenate(labels)
    
    return losses.avg,pred,labels

def val_fn(model,val_loader,criterion):
    device='cuda'
    model.eval()
    model.to(device)
    losses=AverageMeter()
    
    preds=[]
    labels=[]
    for i,(x,y) in tqdm_notebook(enumerate(val_loader),total=len(val_loader)):
        x=x.reshape(-1,1,28,28)
        x=x.float()

        x=x.to(device)
        y=y.to(device)
        

        with torch.no_grad():
            y_preds=model(x)
        loss=criterion(y_preds,y.view(-1,1).flatten())
        
        losses.update(loss.item(),Cfg.val_batch)

        y_preds=torch.nn.functional.softmax(y_preds,dim=1)
        y_preds=torch.argmax(y_preds,dim=1)
        y_preds=y_preds.detach().cpu().numpy()
        y=y.flatten().detach().cpu().numpy()
        preds.append(y_preds)
        labels.append(y)
    pred=np.concatenate(preds)
    labels=np.concatenate(labels)
    
    return losses.avg,pred,labels


def test_fn(model,val_loader):
    device='cuda'
    model.eval()
    model.to(device)
    losses=AverageMeter()
    
    preds=[]
    labels=[]
    for i,(x) in tqdm_notebook(enumerate(val_loader),total=len(val_loader)):
        x=x.reshape(-1,1,28,28)
        x=x.float()
        x=x.to(device)
        
        with torch.no_grad():
            y_preds=model(x)
 

        y_preds=torch.nn.functional.softmax(y_preds,dim=1)
        y_preds=torch.argmax(y_preds,dim=1)
        y_preds=y_preds.detach().cpu().numpy()
        
        preds.append(y_preds)
        
    pred=np.concatenate(preds)
   
    return pred
            

In [ ]:
def prepare_input(df,fold):
    train_df=df[df['fold']!=fold].reset_index(drop=True)
    val_df=df[df['fold']==fold].reset_index(drop=True)
    train_ds=TrainDataset(train_df)
    val_ds=TrainDataset(val_df)
    train_dl=DataLoader(train_ds,batch_size=Cfg.tr_batch)
    val_dl=DataLoader(val_ds,batch_size=Cfg.val_batch)
    train_steps=int(len(train_dl)*Cfg.epochs/Cfg.acc_steps)
    return train_dl,val_dl,train_steps

In [ ]:
def train_loop(fold):
    model=Model()
    criterion=torch.nn.CrossEntropyLoss(reduction='mean')
    optimizer=AdamW(model.parameters(),lr=Cfg.lr,betas=Cfg.betas)
    train_dl,val_dl,num_steps=prepare_input(train_df,fold)
    best_loss=np.inf
    x=0
    l=[]
    for epoch in range(Cfg.epochs):
        tr_loss,pred,labels=train_fn(model,train_dl,criterion,optimizer)
    

        tr_score=sum(np.array(pred)==np.array(labels))/len(labels)
        val_loss,pred,labels=val_fn(model,val_dl,criterion)
        val_score=sum(np.array(pred)==np.array(labels))/len(labels)
        print(f'train and val loss are {tr_loss} and {val_loss}')
        print(f'train and val score are {tr_score} and {val_score}')
        if val_loss<best_loss:
            
            print(f'previous best loss was {best_loss} and new loss is {val_loss}')
            best_loss=val_loss
            torch.save(model,f'model_{fold}_{epoch+1}.bin')
            l.append(epoch+1)
            try:
                os.remove(f'model_{fold}_{l[-2]}.bin')
            except:
                print(f'model not found')
        else:
            x=x+1
        if x==20:
            break
            
        
    

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
from torch import nn

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv=torch.nn.Sequential(
        nn.Conv2d(1,64,padding='same',stride=1,kernel_size=3),
        nn.BatchNorm2d(64),
        nn.ReLU(inplace=True),
        nn.Conv2d(64,64,padding='same',stride=1,kernel_size=3),
        nn.BatchNorm2d(64),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2,stride=2),
        nn.Conv2d(64,128,padding='same',stride=1,kernel_size=3),
        nn.BatchNorm2d(128),
        nn.ReLU(inplace=True),
        nn.Conv2d(128,128,padding='same',stride=1,kernel_size=3),
        nn.BatchNorm2d(128),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2,stride=2),
        nn.Conv2d(128,256,padding='same',stride=1,kernel_size=3),
        nn.BatchNorm2d(256),
        nn.ReLU(inplace=True),
        nn.Conv2d(256,256,padding='same',stride=1,kernel_size=3),
        nn.BatchNorm2d(256),
        nn.ReLU(inplace=True))
        
        self.classifier=nn.Sequential(
        nn.Linear(7*7*256,512),
        nn.BatchNorm1d(512),
        nn.ReLU(inplace=True),
        nn.Dropout(.1),
        nn.Linear(512,512),
        nn.BatchNorm1d(512),
        nn.ReLU(inplace=True),
        nn.Dropout(.1),
        nn.Linear(512,256),
        nn.BatchNorm1d(256),
        nn.ReLU(inplace=True),
        nn.Linear(256,10))
        
        for m in self.conv.children():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                
                m.weight.data.fill_(1)
                m.bias.data.zero_()
                
        for m in self.classifier.children():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform(m.weight)
            elif isinstance(m, nn.BatchNorm1d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
    def forward(self,x):
        x=self.conv(x)
        x=x.view(x.size(0),-1)
        x=self.classifier(x)
        return x
    
        
        

In [ ]:
import math
from tqdm import tqdm_notebook

In [ ]:
for i in range(0,5):
    train_loop(i)